In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling3D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import ModelCheckpoint
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.models import load_model
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


In [18]:
input_shape_ = [101, 108, 1]

In [3]:
#!pip install -U -q PyDrive
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

In [4]:
#link = 'https://drive.google.com/open?id=1ph8OBd6RffqmCXxjjAPgb6PX2jKcB2JF'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('X_accordion.npy')
X_accordion = np.load('X_accordion.npy')

#link = 'https://drive.google.com/open?id=1uLUGKZepXealNHcgE-oH22hyFQd_EVL1'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('Y_accordion.npy')
Y_accordion = np.load('Y_accordion.npy')

In [5]:
#link = 'https://drive.google.com/open?id=1GYwito9726CA56kKYjNyIid5OpWibpIp'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('X_piano.npy')
X_piano = np.load('X_piano.npy')

#link = 'https://drive.google.com/open?id=1VfTNBm3HyaSEaoUuOjC4LyoESrOvtcm4'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('Y_piano.npy')
Y_piano = np.load('Y_piano.npy')

In [6]:
#link = 'https://drive.google.com/open?id=1StsGt8xABBQmLUmyds3oSjcKVBLkaHdA'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('X_violin.npy')
X_violin = np.load('X_violin.npy')

#link = 'https://drive.google.com/open?id=1arptg0ukDIBqpIWkdvl9BpWGskBenHVA'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('Y_violin.npy')  
Y_violin = np.load('Y_violin.npy')

In [7]:
#link = 'https://drive.google.com/open?id=1LqyzrUElRD6QAXYc8CRKiYRtzFFoEhc2'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('X_guitar.npy')  
X_guitar = np.load('X_guitar.npy')

#link = 'https://drive.google.com/open?id=1_u5kzFCitEqslNF0lMW9IeJpwl0zDodG'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('Y_guitar.npy')
Y_guitar = np.load('Y_guitar.npy')

In [8]:
#link = 'https://drive.google.com/open?id=1ikjxpQt04Alcl9lTatZa49HxIJYIm1OD'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('X_noise.npy')
X_noise = np.load('X_noise.npy')

#link = 'https://drive.google.com/open?id=1v_SjZaHsvcNmI0CodMcabd5-iyKLBeOm'
#fluff, id = link.split('=')
#print (id) # Verify that you have everything after '='
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('Y_noise.npy')
Y_noise = np.load('Y_noise.npy')

In [9]:
X = np.concatenate((X_accordion, X_piano, X_violin, X_guitar, X_noise),axis=0)
Y = np.concatenate((Y_accordion, Y_piano, Y_violin, Y_guitar, Y_noise),axis=0)

In [10]:
print(X.shape)
print(Y.shape)

(57504, 101, 108)
(57504, 5)


In [11]:
indices = np.arange(Y.shape[0])
np.random.shuffle(indices)

X = X[indices]
Y = Y[indices]

In [12]:
X_train_orig = X[0:50000]
Y_train = Y[0:50000]
X_test_orig = X[50000:57504]
Y_test = Y[50000:57504]

In [13]:
maxElement = np.amax(X_train_orig)

In [14]:
print(maxElement)

16462066.603519555


In [15]:
# Normalize image vectors
X_train = X_train_orig/maxElement
X_test = X_test_orig/maxElement

In [16]:
train_shape = list(X_train.shape)
train_shape.append(1)
X_train = X_train.reshape(train_shape)

test_shape = list(X_test.shape)
test_shape.append(1)
X_test = X_test.reshape(test_shape)

In [26]:
def Detector(input_shape = input_shape_):
    """
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    
    ### START CODE HERE ###
    # Feel free to use the suggested outline in the text above to get started, and run through the whole
    # exercise (including the later portions of this notebook) once. The come back also try out other
    # network architectures as well. 
        # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((1, 1))(X_input)
    X = Conv2D(32, (3, 3), strides=(1, 1), name='conv0_0')(X)
    #X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)
    X = ZeroPadding2D((1, 1))(X_input)
    X = Conv2D(32, (3, 3), strides=(1, 1), name='conv0_1')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((3, 3), name='max_pool_0')(X)    
    X = Dropout(0.25)(X)
    
    X = ZeroPadding2D((1, 1))(X_input)
    X = Conv2D(64, (3, 3), strides=(1, 1), name='conv1_0')(X)
    #X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)
    X = ZeroPadding2D((1, 1))(X_input)
    X = Conv2D(64, (3, 3), strides=(1, 1), name='conv1_1')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((3, 3), name='max_pool_1')(X)    
    X = Dropout(0.25)(X)

    X = Conv2D(128, (3, 3), strides=(1, 1), name='conv2_0')(X)
    #X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)    
    X = Conv2D(128, (3, 3), strides=(1, 1), name='conv2_1')(X)
    X = Activation('relu')(X)
    
    # Global MAXPOOL
    #X = GlobalMaxPooling2D(data_format=None)(X) 
    
    # MAXPOOL
    #X = MaxPooling2D((3, 3), name='max_pool_2')(X)    
    #X = Dropout(0.25)(X)
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1024, activation=None, name='fc_1024')(X)
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)    
    X = Dense(5, activation='softmax', name='fc_3')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs=X_input, outputs=X, name='Detector')

    return model
    ### END CODE HERE ###
    
    return model

In [27]:
detector = Detector()

In [28]:
detector.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [29]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [34]:
detector.fit(X_train, Y_train, epochs=40, batch_size=50, callbacks=callbacks_list)

Epoch 1/40
50000/50000 [==============================] - 2013s 40ms/step - loss: 0.2393 - acc: 0.8959
Epoch 2/40


/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


50000/50000 [==============================] - 2001s 40ms/step - loss: 0.2124 - acc: 0.9078
Epoch 3/40
 4000/50000 [=>............................] - ETA: 30:36 - loss: 0.1918 - acc: 0.9192

KeyboardInterrupt: 

In [35]:
preds = detector.evaluate(X_test, Y_test, batch_size=32, verbose=1, sample_weight=None)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

7504/7504 [==============================] - 45s 6ms/step

Loss = 0.17789896140728934
Test Accuracy = 0.9228144931132352


In [37]:
detector.save('my_model.h5')